In [81]:
# Importy
# Konfiguracja środowiska i przygotowanie danych
#!pip install sqlalchemy cx_Oracle pandas
#!pip install seaborn
#!pip install matplotlib
#!pip install plotly
#!pip install redis
#!pip install pandas
#!pip install pymongo
from sqlalchemy import create_engine
import sqlalchemy as sa
# Importowanie bibliotek
from datetime import datetime, timedelta
import pandas as pd
from pymongo import MongoClient
import pprint
#import matplotlib.pyplot as plt
#import seaborn as sns
#import plotly.express as px
#from IPython.display import display
import cx_Oracle
import json

# Konfiguracja bazy danych
username = 'S101095'
password = 'S101095'
host = '217.173.198.135'
port = '1521'
service_name = 'tpdb'

dsn = f'{username}:{password}@{host}:{port}/?service_name={service_name}'
 
engine = create_engine(f'oracle+cx_oracle://{username}:{password}@{host}:{port}/?service_name={service_name}')
connection = engine.connect()

# Połączenie z bazą danych

# Połączenie z bazą MongoDB
client = MongoClient("mongodb://localhost:50001/")
print(client.list_database_names()) 
db = client["mongo-server"] 
collection = db["user_data"]  


# Wstawianie danych do MongoDB
# Pobieranie danych z tabel Oracle
query_bmi = "SELECT CalculationDate, ResultBMI, UserName FROM BmiResult"
query_price = "SELECT PriceDate, Price, UserName FROM Price"
query_weight = "SELECT WeightDate, UserWeight, UserName FROM Weight"


df_bmi = pd.read_sql(query_bmi, connection)
df_price = pd.read_sql(query_price, connection)
df_weight = pd.read_sql(query_weight, connection)

# Grupowanie danych według użytkowników i tworzenie struktury dokumentów
users = set(df_bmi["username"]).union(df_price["username"]).union(df_weight["username"])

data = []
for user in users:
    user_data = {"username": user}
    user_data["bmiresult"] = df_bmi[df_bmi["username"] == user][["calculationdate", "resultbmi"]].to_dict(orient="records")
    user_data["price"] = df_price[df_price["username"] == user][["pricedate", "price"]].to_dict(orient="records")
    user_data["weight"] = df_weight[df_weight["username"] == user][["weightdate", "userweight"]].to_dict(orient="records")
    data.append(user_data)

# Wstawianie danych do MongoDB
collection.delete_many({})  # Wyczyść kolekcję, jeśli to konieczne
collection.insert_many(data)

#A. Wyszukiwanie pojedynczych dokumentów
print('A. Wyszukiwanie pojedynczych dokumentów')

print('1. Wyszukiwanie danych użytkownika "test@wp.pl"')
user = collection.find_one({"username": "test123@wp.pl"})
print(user)
print(' ')

print('2. Wyszukiwanie danych o wadze z konkretnego dnia')
search_date = datetime(2024, 4, 3)
start_of_day = search_date
end_of_day = search_date + timedelta(days=1)

weight_entry = collection.find_one(
    {
        "username": "test@wp.pl",
        "weight.weightdate": {"$gte": start_of_day, "$lt": end_of_day}
    },
    {"weight.$": 1, "_id": 0} 
)
print(weight_entry)
print(' ')

print('3. Wyszukiwanie danych BMI dla "test@wp.pl"')
bmi_entry = collection.find_one(
    {"username": "test@wp.pl"},
    {"bmiresult": 1, "_id": 0}  
)
print(bmi_entry)
print(' ')

#B. Wyszukiwanie grup dokumentów
print('B. Wyszukiwanie grup dokumentów')

print('1. Wyszukiwanie wszystkich użytkowników')
users = collection.find({}, {"username": 1})
for user in users:
    print(user)
print(' ')

print('2. Wyszukiwanie użytkowników o BMI powyżej 22.0')
users_with_high_bmi = collection.find({"bmiresult.resultbmi": {"$gt": 22.0}}, {"username": 1})
for user in users_with_high_bmi:
    print(user)
print(' ')

print('3. Wyszukiwanie użytkowników z wagą poniżej 83')
users_with_low_weight = collection.find({"weight.userweight": {"$lt": 83}}, {"username": 1})
for user in users_with_low_weight:
    print(user)
print(' ')

#C. Wyliczenie wartości
print('C. Wyliczenia wartości')

print('1. Obliczanie średniego BMI użytkownika "test@wp.pl"')
user = collection.find_one({"username": "test@wp.pl"})
if user and "bmiresult" in user:
    bmi_values = [entry["resultbmi"] for entry in user["bmiresult"]]
    avg_bmi = sum(bmi_values) / len(bmi_values)
    print(f"Średnie BMI użytkownika 'test@wp.pl': {avg_bmi}")
print(' ')

print('2. Obliczanie maksymalnej wagi wśród wszystkich użytkowników')
max_weight = collection.aggregate([
    {"$unwind": "$weight"},
    {"$group": {"_id": None, "max_weight": {"$max": "$weight.userweight"}}}
])
print(list(max_weight))
print(' ')

print('3. Liczba wpisów dla każdego użytkownika')
entry_counts = collection.aggregate([
    {"$project": {"username": 1, "total_entries": {"$size": {"$concatArrays": ["$weight", "$bmiresult", "$price"]}}}}
])
for count in entry_counts:
    print(count)
print(' ')

#D. Agregacja wyników
print('D. Agregacje wyników')

print('1. Suma wszystkich wpisów dla wszystkich użytkowników')
total_weight = collection.aggregate([
    {"$project": {"count": {"$size": {"$ifNull": ["$weight", []]}}}},
    {"$group": {"_id": None, "total_weight": {"$sum": "$count"}}}
])

total_bmi = collection.aggregate([
    {"$project": {"count": {"$size": {"$ifNull": ["$bmiresult", []]}}}},
    {"$group": {"_id": None, "total_bmi": {"$sum": "$count"}}}
])

total_price = collection.aggregate([
    {"$project": {"count": {"$size": {"$ifNull": ["$price", []]}}}},
    {"$group": {"_id": None, "total_price": {"$sum": "$count"}}}
])
print(f'Total weight entries: {list(total_weight)}')
print(f'Total BMI entries: {list(total_bmi)}')
print(f'Total price entries: {list(total_price)}')
print('')

print('2. Liczba użytkowników w bazie')
user_count = collection.distinct("username")
print(f"Liczba użytkowników: {len(user_count)}")
print(' ')

print('3. Średnia cena z bazy')
avg_price = collection.aggregate([
    {"$unwind": "$price"},
    {"$group": {"_id": None, "avg_price": {"$avg": "$price.price"}}}
])
print(list(avg_price))



['admin', 'config', 'local', 'mongo-server']
A. Wyszukiwanie pojedynczych dokumentów
1. Wyszukiwanie danych użytkownika "test@wp.pl"
{'_id': ObjectId('678534f8691911714162cb17'), 'username': 'test123@wp.pl', 'bmiresult': [{'calculationdate': datetime.datetime(2024, 3, 10, 23, 52, 4, 653000), 'resultbmi': 23.0}, {'calculationdate': datetime.datetime(2024, 3, 11, 10, 25, 53, 410000), 'resultbmi': 22.0}], 'price': [], 'weight': [{'weightdate': datetime.datetime(2024, 3, 11, 10, 22, 18, 343000), 'userweight': 80.0}, {'weightdate': datetime.datetime(2024, 3, 11, 10, 22, 23, 233000), 'userweight': nan}]}
 
2. Wyszukiwanie danych o wadze z konkretnego dnia
{'weight': [{'weightdate': datetime.datetime(2024, 4, 3, 17, 14, 30, 323000), 'userweight': 85.0}]}
 
3. Wyszukiwanie danych BMI dla "test@wp.pl"
{'bmiresult': [{'calculationdate': datetime.datetime(2024, 2, 28, 13, 48, 47, 70000), 'resultbmi': 21.2}, {'calculationdate': datetime.datetime(2024, 3, 1, 17, 55, 40, 387000), 'resultbmi': 24.38}